<a href="https://colab.research.google.com/github/abisatyaw/Tensorflow2Coursera/blob/main/Exercise_4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [11]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 100% Accuracy!!")
      self.model.stop_training = True


callbacks = myCallback()

--2021-04-06 10:22:54--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.195.128, 64.233.191.128, 209.85.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-04-06 10:22:54 (157 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [12]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
        # The First convolution
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
        tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=0.001),
                  metrics=['accuracy'])

In [13]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 300x300
        batch_size=128,
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [27]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
          train_generator,
          batch_size = 120,  
          epochs=15,
          verbose=1,
          callbacks=[callbacks]
)
    # model fitting
#return history.history['acc'][-1]
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
1/1 [==============================] - 2s 2s/step - loss: 0.5866 - accuracy: 0.5000
Epoch 2/15
1/1 [==============================] - 2s 2s/step - loss: 0.4834 - accuracy: 0.6875
Epoch 3/15
1/1 [==============================] - 2s 2s/step - loss: 0.3689 - accuracy: 0.9125
Epoch 4/15
1/1 [==============================] - 2s 2s/step - loss: 0.5484 - accuracy: 0.6625
Epoch 5/15
1/1 [==============================] - 2s 2s/step - loss: 0.7143 - accuracy: 0.5000
Epoch 6/15
1/1 [==============================] - 2s 2s/step - loss: 0.3667 - accuracy: 0.8875
Epoch 7/15
1/1 [==============================] - 2s 2s/step - loss: 0.2306 - accuracy: 0.9250
Epoch 8/15
1/1 [==============================] - 2s 2s/step - loss: 0.1774 - accuracy: 0.9875
Epoch 9/15
1/1 [==============================] - 2s 2s/step - loss: 0.1538 - accuracy: 0.9375
Epoch 10/15
1/1 [==============================] - 2s 2s/step - loss: 0.1304 - accuracy: 1.0000

Reached 100% Accuracy!!
